In [6]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import xarray as xr
import numpy as np
import pickle

from glob import glob
from minghao_utils import makedir
from minghao_utils import seed_everything

XR_DATA_PATH = "/l/users/minghao.fu/dataset/CESM2/CESM2_pacific_grouped_SST.nc"
SPACE_INDEX_DATA_PATH = "/l/users/minghao.fu/dataset/CESM2/CESM2_pacific.pkl"
PATH = '/home/minghao.fu/workspace/climate/tests/results'
seed_everything(42)

xr_data = xr.open_dataset(XR_DATA_PATH)

CESM2_adj_mats = []
for idx, path in enumerate(glob(f'{PATH}/adj_mats/*.npy')):
       CESM2_adj_mats.append(np.load(path))

CESM2_treatment_effects = np.load(f'{PATH}/treatment_effects.npy')
CESM2_coords = np.load(f'{PATH}/coords.npy')


In [11]:
# hyper parameter
TIME_IDX = 0


In [12]:

# plot treatment effects
makedir('./files/treatment_effect/')

normalized_effects = np.zeros_like(CESM2_treatment_effects)
for i in range(CESM2_treatment_effects.shape[0]):
    for j in range(CESM2_treatment_effects.shape[1]):
        normalized_effects[i, j, :] = CESM2_treatment_effects[i, j, :] / np.linalg.norm(CESM2_treatment_effects[i, j, :])

for z_idx in range(CESM2_treatment_effects.shape[1]):
    data = traindata.xr_data.copy().isel(time=TIME_IDX)
    for idx, agroup_index in enumerate(CESM2_group_indices):
        mask = train_data.xr_ds.group_index == agroup_index
        # Assign values based on the mask
        data.values[mask] = normalized_effects[TIME_IDX, z_idx, idx]  # Assuming 'effects_lst' is properly defined
        
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree(-120)})
    #plt.subplots_adjust(left=0.1, right=0.2, top=0.2, bottom=0.1, wspace=0.2, hspace=0.2)
    data.plot(x='lon', y='lat', ax=ax)
    fig.savefig(f'./files/treatment_effect/z{z_idx}.png', dpi=500)
    save_path=f"/l/users/minghao.fu/dataset/CESM2/CESM2_z{z_idx}_treatment_effects.nc"
    grouped_sst_data.to_netcdf(save_path)
    grouped_sst_data

NameError: name 'CESM2_group_indices' is not defined

In [14]:
if PLOT_HEATMAP:
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree(-120)})
    plt.subplots_adjust(left=0.1, right=0.2, top=0.2, bottom=0.1, wspace=0.2, hspace=0.2)
    ax.stock_img()
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude')
    ax.set_title('Sea Surface Temperature (SST)')
    gl = ax.gridlines(draw_labels=True, color='black', alpha=0.5, linestyle='--')
    gl.top_labels = False  # Disable labels at the top
    gl.right_labels = False  # Disable labels on the right
    gl.xlabel_style = {'size': 12}
    gl.ylabel_style = {'size': 12}
    # temperature heatmap of time=0
    xr_data.isel(time=TIME_IDX).plot(x='lon', y='lat', ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False)
    fig.savefig(f'./files/heatmap.png', dpi=500)

if PLOT_DAG:
    for idx, adj_mat in enumerate(CESM2_adj_mats):
        for i in range(adj_mat.shape[0]):
            for j in range(adj_mat.shape[1]):
                if np.abs(adj_mat[i, j]) > DAG_THRES:
                    start_coords = CESM2_coords[idx * adj_mat.shape[0] + j]
                    end_coords = CESM2_coords[idx * adj_mat.shape[0] + i]
                    ax.plot(start_coords[1], start_coords[0],
                        color='blue', markersize=1, marker='o', zorder=0,
                        transform=ccrs.PlateCarree(),
                        )
                    ax.plot(end_coords[1], end_coords[0],
                        color='blue', markersize=1, marker='o', zorder=0,
                        transform=ccrs.PlateCarree(),
                        )
                    # u = end_coords[1] - start_coords[1]  # Longitude difference
                    # v = end_coords[0] - start_coords[0]  # Latitude difference
                    # ax.quiver(start_coords[1], start_coords[0], u, v, color='blue', scale_units='xy', angles='xy', scale=1, transform=ccrs.PlateCarree())
                    ax.annotate('', xy=(end_coords[1], end_coords[0]), xytext=(start_coords[1], start_coords[0]), zorder=5,
                                xycoords='data', textcoords='data', markeredgecolor='blue', markerfacecolor='none',
                                arrowprops=dict(arrowstyle="->, head_width=0.1, head_length=0.1", color='red', lw=0.5, connectionstyle="arc3"),
                                transform=ccrs.PlateCarree())

    fig.savefig(f'./CESM2/supper_B_{TIME_IDX}/area.png', dpi=2000)




In [18]:
ds = xr_ds.isel(time=0)
ds.values[ds.group_index == 182] = 0

In [21]:
ds.group_index

<xarray.DataArray 'group_index' (nlat: 186, nlon: 151)>
array([[569., 569., 569., ..., 247., 247., 247.],
       [569., 569., 569., ..., 247., 247., 247.],
       [569., 569., 569., ..., 247., 247., 247.],
       ...,
       [897., 897., 897., ..., 726., 726., 726.],
       [897., 897., 897., ..., 726., 726., 726.],
       [897., 897., 897., ..., 726., 726., 726.]])
Coordinates:
  * nlat         (nlat) int32 94 95 96 97 98 99 100 ... 274 275 276 277 278 279
  * nlon         (nlon) int32 143 144 145 146 147 148 ... 289 290 291 292 293
    lat          (nlat, nlon) float64 ...
    lon          (nlat, nlon) float64 ...
    group_index  (nlat, nlon) float64 569.0 569.0 569.0 ... 726.0 726.0 726.0
    area_index   (nlat, nlon) float64 ...